# Colab Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import lib

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from pathlib import Path

import warnings
warnings.filterwarnings(action='ignore') 

# CUDA, Colab path, Config, Random seed 설정 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
colab_path = '/content/drive/MyDrive/KNU_DATASCIENCE/공모전/DACON_TV손동작'

In [5]:
CFG = {
    'FPS':30,
    'IMG_SIZE':128,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41,
    'nfold':5,
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [8]:
# local일 경우 이 코드 사용
#df = pd.read_csv('./train.csv')

# colab
df = pd.read_csv(colab_path + '/train.csv')
df['path'] = colab_path + df['path'].str[1:]

In [9]:
df

,id,path,label
0,TRAIN_000,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,3
1,TRAIN_001,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,0
2,TRAIN_002,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,1
3,TRAIN_003,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,4
4,TRAIN_004,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,4
...,...,...,...
605,TRAIN_605,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,0
606,TRAIN_606,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,2
607,TRAIN_607,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,1
608,TRAIN_608,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...,4


In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

# Code

In [11]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list, transform=None):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.transform = transform
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['FPS']):
            _, img = cap.read()
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            img = img / 255.
            frames.append(img)

        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [12]:
train_dataset = CustomDataset(train['path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [13]:
def model_train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    train_loss_epoch, valid_loss_epoch, valid_score_epoch = [], [], []
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _train_loss = np.mean(train_loss)
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        train_loss_epoch.append(_train_loss)
        valid_loss_epoch.append(_val_loss)
        valid_score_epoch.append(_val_score)
        

        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            print('***** Best Model *****')
            
    return best_model, train_loss_epoch, valid_loss_epoch, valid_score_epoch

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

# Run

In [16]:
model = models.video.r3d_18(pretrained = True)
model.fc = nn.Linear(in_features=512, out_features=5)

Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


  0%|          | 0.00/127M [00:00<?, ?B/s]

In [17]:

model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model, train_loss_epoch_, valid_loss_epoch_, valid_score_epoch_ = model_train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.64176] Val Loss : [0.24670] Val F1 : [0.90476]
***** Best Model *****


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [7], Train Loss : [0.06705] Val Loss : [0.02182] Val F1 : [1.00000]


  0%|          | 0/122 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

---

# Inference

In [18]:
# local일 경우 이 코드 사용
#test = pd.read_csv('/test.csv')
#test['path'] = test['path'].str[1:]

# colab
test = pd.read_csv(colab_path + '/test.csv')
test['path'] = colab_path + test['path'].str[1:]
test

,id,path
0,TEST_000,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
1,TEST_001,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
2,TEST_002,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
3,TEST_003,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
4,TEST_004,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
...,...,...
148,TEST_148,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
149,TEST_149,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
150,TEST_150,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...
151,TEST_151,/content/drive/MyDrive/KNU_DATASCIENCE/공모저...


In [19]:
test_dataset = CustomDataset(test['path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [20]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [21]:
preds = inference(model, test_loader, device)

  0%|          | 0/39 [00:00<?, ?it/s]

In [22]:
# local일 경우 이 코드 사용
#submit = pd.read_csv('/sample_submission.csv')

# colab
submit = pd.read_csv(colab_path + '/sample_submission.csv')

In [ ]:
submit['label'] = preds
submit.head()

,id,label
0,TEST_000,1
1,TEST_001,3
2,TEST_002,0
3,TEST_003,2
4,TEST_004,4


In [ ]:
# local일 경우 이 코드 사용
#submit.to_csv('/r3d_18_submit.csv', index=False)

# colab
submit.to_csv(colab_path +  '/r3d_18_submit.csv', index=False)